In [1]:
import pandas as pd

# Criação de vocabulário

Percebemos que as palavras chave presentes na coluna `keywords` não correspondem ao vocabulário presente no conjunto de dados "Temas" presente no Portal de Dados Abertos da Câmara dos Deputados Federais e coletado pelo notebook [../coleta](../coleta). Por esse motivo foi necessário correlacionar as palavras chave presentes na coluna `keywords` com os temas do vocabulário.

In [2]:
proposicoes = pd.read_csv('../dados/proposicoes_legislativas_limpas.csv')

In [3]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152386 entries, 0 to 152385
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               152386 non-null  int64 
 1   siglaTipo        152386 non-null  object
 2   ano              152386 non-null  int64 
 3   codTipo          152386 non-null  int64 
 4   descricaoTipo    152386 non-null  object
 5   ementa           152386 non-null  object
 6   ementaDetalhada  42563 non-null   object
 7   keywords         152386 non-null  object
dtypes: int64(3), object(5)
memory usage: 9.3+ MB


Reunião de palavras chaves para classificação

In [4]:
keywords = proposicoes['keywords']

In [5]:
vocabulario = []

In [6]:
for proposicao in keywords:
    lista = proposicao.split(',')
    vocabulario.extend(lista)

In [7]:
vocabulario_unico = set(vocabulario)

In [8]:
with open('../dados/vocabulario.txt', 'w') as palavras:
        for termo in vocabulario_unico:
            palavras.write(termo + '\n')

Foram revisadas as palavras chaves contidas no arquivo anterior e selecionadas apenas as referentes a proteção de direitos de grupos historicamente marginalizados. Em seguida, as palavras foram relacionadas manualmente aos temas presentes no conjunto de dados "Temas" presente no Portal de Dados Abertos da Câmara dos Deputados Federais.

In [9]:
vocabulario_temp = pd.read_csv('../dados/temas_vocabulario.csv')

In [10]:
vocabulario_temp.head()

,cod,sigla,nome,descricao
0,34,NaN,Administração Pública,NaN
1,64,NaN,"Agricultura, Pecuária, Pesca e Extrativismo",NaN
2,35,NaN,"Arte, Cultura e Religião",territporios obra-de-arte histórico-cultural a...
3,41,NaN,Cidades e Desenvolvimento Urbano,natalidade pacificação
4,62,NaN,"Ciência, Tecnologia e Inovação",NaN


Cria vocabulário completo

In [11]:
vocabulario = pd.DataFrame(columns=['cod', 'tema', 'palavra_chave'])

In [12]:
indices = vocabulario_temp.index

In [13]:
for indice in indices:
    descricao = vocabulario_temp['descricao'].iloc[indice]
    if type(descricao) == str:
        for palavra in descricao.split(' '):
            df = pd.DataFrame(data={'cod':vocabulario_temp['cod'].iloc[indice], 'tema':vocabulario_temp['nome'].iloc[indice], 'palavra_chave':[palavra]})
            vocabulario = pd.concat([vocabulario, df], ignore_index=True)
        

In [14]:
vocabulario.head()

,cod,tema,palavra_chave
0,35,"Arte, Cultura e Religião",territporios
1,35,"Arte, Cultura e Religião",obra-de-arte
2,35,"Arte, Cultura e Religião",histórico-cultural
3,35,"Arte, Cultura e Religião",aleijadinho
4,35,"Arte, Cultura e Religião",ancine


In [15]:
vocabulario.shape

(109, 3)

Retira palavras chaves em branco, caso haja

In [16]:
vocabulario = vocabulario[vocabulario['palavra_chave']!= ''].copy()
vocabulario.shape

(109, 3)

Atribuir o tema às proposições legislativas, apenas aquelas cuja temática refer-se a proteção de direitos de grupos historicamente marginalizados.

In [17]:
def atribui_tema(proposicao):
    for tema, palavra_chave in zip(vocabulario['tema'], vocabulario['palavra_chave']):
        if palavra_chave in proposicao:
            return tema

In [18]:
proposicoes['temas'] = proposicoes['keywords'].apply(atribui_tema)

In [19]:
proposicoes.to_csv('../dados/proposicoes_legislativas_limpas_vocabulario.csv', index=False)